# Treinamento do modelo

Dataset: https://www.kaggle.com/datasets/dennisho/blackjack-hands/

Modelo: MLP (sem o uso de card counting)

## 1) Tratamento dos dados

In [8]:
import pandas as pd

rows = 60000
df = pd.read_csv('blackjack_simulator.csv', sep=',', nrows=rows)

# Remove as colunas não utilizadas
df = df.drop(columns=['shoe_id', 'run_count', 'true_count', 'cards_remaining', 'dealer_up', 'initial_hand', 'dealer_final_value', 'player_final_value'])

# Filtra os dados por ganho
df = df[df['win'] >= 1]

# Transforma as strings em listas
df['dealer_final'] = df['dealer_final'].apply(lambda x: eval(x)[0])  # Só a primeira carta do dealer será utilizada para emular as informações que um jogador tem
df['player_final'] = df['player_final'].apply(lambda x: eval(x))
df['actions_taken'] = df['actions_taken'].apply(lambda x: eval(x))
df

,dealer_final,player_final,actions_taken,win
0,10,"[[10, 11]]",[[S]],1.5
1,10,"[[5, 5, 11]]","[[H, S]]",1.0
2,6,"[[3, 10]]",[[S]],1.0
6,6,"[[3, 2, 10], [3, 4, 5]]","[[P, H, S], [H, S]]",2.0
8,5,"[[8, 10]]",[[S]],1.0
...,...,...,...,...
59991,11,"[[2, 8, 3, 7]]","[[N, H, H, S]]",1.0
59992,9,"[[10, 9]]",[[S]],1.0
59993,4,"[[7, 3, 10]]",[[D]],2.0
59994,10,"[[9, 10]]",[[S]],1.0


In [9]:
# Divide as ações dos players em vários estados de jogo
for i in range(0, len(df.index)):
    linha = df.loc[i]

    for j in range(0, len(linha['actions_taken'])):
        mao = linha['player_final'][j]
        acoes = linha['actions_taken'][j]

        # print({'dealer_final': linha['dealer_final'], 'player_final': mao[:4], 'actions_taken': acoes[0]})

        for k in range(0, len(acoes)):
            acao = acoes[k]

            df = pd.concat([df, pd.DataFrame([{'dealer_final': linha['dealer_final'], 'player_final': sum(mao[:2+k]), 'actions_taken': acoes[k], 'win': linha['win']}])], ignore_index=True)

df = df.drop(range(0, rows))
df


,dealer_final,player_final,actions_taken,win
60000,6,18,D,2.0
60001,7,7,H,1.0
60002,7,17,S,1.0
60003,10,11,D,2.0
60004,6,10,D,2.0
...,...,...,...,...
62574,11,20,S,1.0
62575,9,19,S,1.0
62576,4,10,D,2.0
62577,10,19,S,1.0


## 2) Treinamento

In [32]:
# from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

X = df.drop(['actions_taken'], axis=1)
y = df['actions_taken']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# clf = MLPClassifier().fit(X_train, y_train)
# clf.score()
clf = LogisticRegression(max_iter=2000, random_state=0).fit(X_train[['dealer_final', 'player_final']], y_train, sample_weight=X_train['win'].values)
clf.score(X_test.drop('win', axis=1), y_test)

0.7810077519379846

### Probabilidade de realizar cada jogada

In [33]:
# Estado do jogo
X_test = X_test.drop('win', axis=1)
X_test[:1]

,dealer_final,player_final
60881,7,5


In [34]:
# Probabilidade de realizar cada jogada
classes = clf.classes_
probabilidade = clf.predict_proba(X_test[:1])[0]

for i in range(0, len(classes)):
    print(f'JOGADA: {classes[i]} {probabilidade[i]*100.0:.02f} %')

JOGADA: D 44.57 %
JOGADA: H 51.38 %
JOGADA: N 0.00 %
JOGADA: P 2.86 %
JOGADA: S 1.19 %


In [35]:
# Jogada a ser realizada
clf.predict(X_test[:1])[0]

'H'

## 3) Exportação

In [7]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(clf, f)